In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
import pandas as pd
import time as t
import statistics


2021-10-22 19:09:08.821411: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Loading the MSMARCO Dataset
MSMARCO = pd.read_csv("MSMARCO_biomedical_PT/test.csv")[["question_pt"]]

# Cleaning MSMARCO Dataset
MSMARCO['question_pt'] = MSMARCO['question_pt'].apply(lambda x: x.rstrip('\n'))
MSMARCO['question_pt'] = MSMARCO['question_pt'].apply(lambda x: x.rstrip('\t'))
MSMARCO['question_pt'] = MSMARCO['question_pt'].apply(lambda x: x.rstrip('?'))
MSMARCO['question_pt'] = MSMARCO['question_pt'].apply(lambda x: x.rstrip('.'))

#MSMARCO.tail(20)



# Loading the MIMIC Dataset
MIMIC = pd.read_csv("MIMICSQL_PT/test.csv")[["question_pt"]]


# Cleaning MIMIC Dataset
MIMIC['question_pt'] = MIMIC['question_pt'].apply(lambda x: x.rstrip('\n'))
MIMIC['question_pt'] = MIMIC['question_pt'].apply(lambda x: x.rstrip('\t'))
MIMIC['question_pt'] = MIMIC['question_pt'].apply(lambda x: x.rstrip('?'))
MIMIC['question_pt'] = MIMIC['question_pt'].apply(lambda x: x.rstrip('.'))



# Loading the SPIDER Dataset
SPIDER = pd.read_csv("SPIDER/hospital_1.txt", sep=";;", header=None)
SPIDER.columns = ["question_pt"]

# Cleaning SPIDER Dataset
SPIDER['question_pt'] = SPIDER['question_pt'].apply(lambda x: x.rstrip('\n'))
SPIDER['question_pt'] = SPIDER['question_pt'].apply(lambda x: x.rstrip('\t'))
SPIDER['question_pt'] = SPIDER['question_pt'].apply(lambda x: x.rstrip('?'))
SPIDER['question_pt'] = SPIDER['question_pt'].apply(lambda x: x.rstrip('.'))

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
#Join datasets


MSMARCO["type"] = 1
MIMIC["type"] = 0
SPIDER["type"] = 0


tp0 = pd.concat([MIMIC, SPIDER], ignore_index=True)

df = pd.concat([MSMARCO, tp0], ignore_index = True)

All test Dataset

In [4]:
# Defining the model
model = ClassificationModel('bert', 'Bertimbau_Classificador/checkpoint-2845-epoch-5', num_labels=2)


# Validate the model
result, model_outputs, wrong_predictions = model.eval_model(df)
preds, _ = model.predict(list(df['question_pt']))

acc = accuracy_score(df['type'].values, preds)
print("accuracy_score: " + str(acc))
mcc = matthews_corrcoef(df['type'].values, preds)
f1_weighted = f1_score(df['type'].values, preds, average='weighted')
f1_micro = f1_score(df['type'].values, preds, average='micro')
f1_macro = f1_score(df['type'].values, preds, average='macro')

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1401: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/5843 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/731 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/5843 [00:00<?, ?it/s]

  0%|          | 0/731 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


accuracy_score: 0.9974328256032859


MSMARCO

In [5]:
# Validate the model
result, model_outputs, wrong_predictions = model.eval_model(MSMARCO)
preds, _ = model.predict(list(MSMARCO['question_pt']))

acc = accuracy_score(MSMARCO['type'].values, preds)
print("accuracy_score: " + str(acc))

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1401: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4743 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/593 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


  0%|          | 0/4743 [00:00<?, ?it/s]

  0%|          | 0/593 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


accuracy_score: 0.9981024667931688


In [13]:
MSMARCO['pred'] = pd.Series(preds, index=MSMARCO.index)
pd.set_option("display.max_colwidth", None)

MSMARCO[MSMARCO.pred == 0]

,question_pt,type,pred
313,o paciente que era um mineiro de carvão,1,0
505,A (n) ____ é o atributo na entidade SuperType que determina a qual subtipo de entidade cada ocorrência de SuperType está relacionada,1,0
1304,O que os medicamentos são chamados de pacientes com saúde mental,1,0
1306,Qual é o propósito do pedido de Hektoen Enteric Agar,1,0
1739,"_____ Descobri diferenças significativas entre os organismos originalmente categorizados sob o Reino Monera; Assim, esses organismos foram colocados em dois domínios separados",1,0
2629,Qual é o propósito do programa de assistência de medicamentos,1,0
2939,que tipo de droga é suramin,1,0
3166,Que opção de tratamento a enfermeira antecipa para o paciente com doença de imunodeficiência combinada grave (SCID),1,0
3867,Qual é o nome da molécula de destino à qual uma enzima persianas,1,0


MIMIC

In [7]:
# Validate the model
result, model_outputs, wrong_predictions = model.eval_model(MIMIC)
preds, _ = model.predict(list(MIMIC['question_pt']))

acc = accuracy_score(MIMIC['type'].values, preds)
print("accuracy_score: " + str(acc))

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1401: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:949: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


accuracy_score: 0.996


In [8]:
MIMIC['pred'] = pd.Series(preds, index=MIMIC.index)
MIMIC[MIMIC.pred == 1]


,question_pt,type,pred
242,Qual idioma cynthia gomez entende,0,1
271,Como a nitroglicerina SL administrada,0,1
356,Theresa vedovelli está sofrendo principalmente...,0,1
397,Como os Nephrocaps são administrados,0,1


SPIDER

In [9]:
# Validate the model
result, model_outputs, wrong_predictions = model.eval_model(SPIDER)
preds, _ = model.predict(list(SPIDER['question_pt']))

acc = accuracy_score(SPIDER['type'].values, preds)
print("accuracy_score: " + str(acc))

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1401: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:949: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


accuracy_score: 0.98


In [10]:
SPIDER['pred'] = pd.Series(preds, index=SPIDER.index)
SPIDER[SPIDER.pred == 1]

,question_pt,type,pred
73,Quais são os três procedimentos mais caros,0,1
77,Qual médico foi treinado no procedimento que c...,0,1
